In [1]:
#playing with https://www.kaggle.com/c/demand-forecasting-kernels-only
#todo
#-one-hot tuning

In [ ]:
import numpy as np
import pandas as pd

In [9]:
data = pd.read_csv('train_sales.csv')

In [15]:
data.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [11]:
data.shape

(913000, 4)

In [12]:
#10개 스토어, 50개 아이템, 2013-01-01 부터 18260 days, sales
data['store'].value_counts()

10    91300
9     91300
8     91300
7     91300
6     91300
5     91300
4     91300
3     91300
2     91300
1     91300
Name: store, dtype: int64

In [50]:
t = '2013-01-01'

In [34]:
t[0].split('-')

['2013', '01', '01']

In [49]:
from datetime import datetime
#a = data['date'].str.split('-')

In [56]:
datetime.strptime(t, '%Y-%m-%d').year

2013

In [64]:
data['year'] = pd.to_datetime(data['date']).dt.year
data['month'] = pd.to_datetime(data['date']).dt.month
data['day'] = pd.to_datetime(data['date']).dt.day

In [65]:
data.head()

,date,store,item,sales,year,month,day
0,2013-01-01,1,1,13,2013,1,1
1,2013-01-02,1,1,11,2013,1,2
2,2013-01-03,1,1,14,2013,1,3
3,2013-01-04,1,1,13,2013,1,4
4,2013-01-05,1,1,10,2013,1,5


In [66]:
data_input = data[['year', 'month', 'day', 'store', 'item']]
data_label = data['sales']

In [68]:
data_input.head()

,year,month,day,store,item
0,2013,1,1,1,1
1,2013,1,2,1,1
2,2013,1,3,1,1
3,2013,1,4,1,1
4,2013,1,5,1,1


In [160]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical

In [91]:
def one_hot_encode(data, features, std=None):
    if std is None:
        res = to_categorical(data, num_classes=features)
    else:
        data_conv = data - std
        res = to_categorical(data_conv, num_classes=features)
    return res

In [98]:
data_encoded = data_input.copy()

In [107]:
year = one_hot_encode(data['year'], len(data['year'].unique())+1, std=2013)
month = one_hot_encode(data['month'], len(data['month'].unique())+1)
day = one_hot_encode(data['day'], len(data['day'].unique())+1)
store = one_hot_encode(data['store'], len(data['store'].unique())+1)
item = one_hot_encode(data['item'], len(data['item'].unique())+1)

In [106]:
one_hot_encode(data['year'], len(data['year'].unique())+1, std=2013)

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [124]:
data_coded = np.concatenate([year,month,day,store,item], axis=1)

In [125]:
data_coded.shape

(913000, 113)

In [189]:
model = Sequential()
model.add(Dense(113, input_shape=(113,)))
for i in range(5):
    model.add(Dense(20, activation='relu'))
    #model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))
#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [190]:
model.fit(train_input, train_label, batch_size=512, epochs=10, verbose=1, validation_split=0.1)

Train on 821610 samples, validate on 91290 samples
Epoch 1/10
821610/821610 [==============================] - 7s 9us/step - loss: 240.6905 - mean_absolute_error: 9.9943 - val_loss: 385.5312 - val_mean_absolute_error: 16.1399
Epoch 2/10
821610/821610 [==============================] - 6s 7us/step - loss: 109.8815 - mean_absolute_error: 7.9144 - val_loss: 386.6285 - val_mean_absolute_error: 16.1842
Epoch 3/10
821610/821610 [==============================] - 6s 7us/step - loss: 104.4594 - mean_absolute_error: 7.6631 - val_loss: 383.3494 - val_mean_absolute_error: 16.2245
Epoch 4/10
821610/821610 [==============================] - 6s 7us/step - loss: 80.1112 - mean_absolute_error: 6.7125 - val_loss: 347.0384 - val_mean_absolute_error: 15.5804
Epoch 5/10
821610/821610 [==============================] - 6s 7us/step - loss: 58.3437 - mean_absolute_error: 5.8587 - val_loss: 349.0948 - val_mean_absolute_error: 15.7064
Epoch 6/10
821610/821610 [==============================] - 6s 7us/step - lo

In [184]:
#원핫을 하고 한번 안하고 한번 해보자 -로스만 높음
#validation도.. -
train_input, test_input = data_coded[:-100], data_coded[-100:]
train_label, test_label = data_label[:-100], data_label[-100:]

In [193]:
prediction = model.predict(test_input)

array([[69.28128 ],
       [74.20982 ],
       [47.74551 ],
       [55.92093 ],
       [57.221344],
       [60.459805],
       [65.027534],
       [69.75589 ],
       [69.038895],
       [44.15879 ],
       [50.969494],
       [53.447784],
       [57.27062 ],
       [59.766926],
       [63.875305],
       [68.29971 ],
       [43.050163],
       [50.553123],
       [53.449306],
       [56.208874],
       [59.66091 ],
       [64.193   ],
       [68.50417 ],
       [43.569042],
       [51.277008],
       [52.765102],
       [56.701412],
       [58.860596],
       [64.33486 ],
       [68.44015 ],
       [43.018894],
       [50.641388],
       [53.855965],
       [56.703583],
       [59.011337],
       [64.16547 ],
       [68.94859 ],
       [42.907043],
       [50.24595 ],
       [58.062553],
       [62.378357],
       [67.50162 ],
       [72.99151 ],
       [71.365456],
       [53.141   ],
       [55.93279 ],
       [57.562775],
       [62.674656],
       [66.70449 ],
       [73.220856],


In [194]:
test_label

912900     97
912901     99
912902     63
912903     79
912904     80
912905     82
912906     90
912907    103
912908     99
912909     71
912910     77
912911     87
912912     78
912913     85
912914    113
912915     91
912916     69
912917     69
912918     69
912919     84
912920     81
912921     97
912922    107
912923     77
912924     68
912925     73
912926     72
912927     78
912928     78
912929     96
         ... 
912970     49
912971     75
912972     54
912973     68
912974     60
912975     66
912976     67
912977     69
912978     69
912979     54
912980     67
912981     67
912982     72
912983     72
912984     52
912985     86
912986     53
912987     54
912988     51
912989     63
912990     75
912991     70
912992     76
912993     51
912994     41
912995     63
912996     59
912997     74
912998     62
912999     82
Name: sales, Length: 100, dtype: int64